In [1]:
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from scipy import ndimage
from PIL import Image
from sklearn.cluster import KMeans
from sklearn import linear_model
from sklearn import svm
from sklearn import preprocessing as prp
from helpers_img_my import *
from Post_processing import *
#from helpers_img import *
from sklearn import model_selection as modsel
from preprocessing_my import *

In [2]:
def post_processing(label,threshold,size_min,verbarg,horbarg,size_image):
    label = complete_lines(label,threshold)
    label = remove_isolated_connected_component(label,size_min)
    label = clean_garbage_vert(label,verbarg,size_image)
    label = clean_garbage_hor(label,horbarg,size_image)
    label = remove_isolated_connected_component(label,size_min)
    #label = complete_semilines(label,0.85, size_image)
    return label

In [ ]:
# Settings
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
patch_size = 16 # each patch is 16*16 pixels
deg=2

root_dir = "training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"
files = os.listdir(image_dir)

n=len(files)
#n=4
k_fold=2 #number of folds in the k-fold cross-validation

ims = [load_image(image_dir + files[i]) for i in range(n)]
gt_ims = [load_image(gt_dir + files[i]) for i in range(n)]

seed = 1
np.random.seed(seed)

kf=modsel.KFold(n_splits=k_fold,shuffle=True)
kf.get_n_splits(ims)
splits=kf.split(ims)
    
ims=[add_features(ims[i]) for i in range(len(ims))]
ims,gt_ims = rotation(ims,gt_ims)
ims,gt_ims = flip(ims,gt_ims)
print(len(ims))

lambdas=np.logspace(-7,-6,1) # 1/lambda
thresh=np.linspace(0.25,0.35,3)
shape=(lambdas.shape[0],thresh.shape[0])
mean_f1=np.zeros(shape)

#for (train_id,test_id) in splits:
trains=[]
tests=[]
for (train_id,test_id) in splits:
    trains.append(train_id)
    tests.append(test_id)
    
print(trains)
print(tests)

for id_lam,lam in enumerate(lambdas):
    print('lambda: '+str(id_lam))
    for id_t,t in enumerate(thresh):
        print('thresh: '+str(id_t))
        nb_f1_te = np.zeros(k_fold)
        ind=0
        #for (train_id,test_id) in splits:
        for train,test in zip(trains,tests):
            print('here')
            print(train)
            print(test)
            im_tr=[ims[idx+n*k] for idx in train for k in np.arange(0,8)]
            im_te=[ims[idx] for idx in test]
            gt_tr=[gt_ims[idx+n*k] for idx in train for k in np.arange(0,8)]
            gt_te=[gt_ims[idx] for idx in test]
            
            im_tr = [img_crop(im_tr[i], patch_size, patch_size) for i in range(len(im_tr))]
            im_te = [img_crop(im_te[i], patch_size, patch_size) for i in range(len(im_te))]
            gt_tr = [img_crop(gt_tr[i], patch_size, patch_size) for i in range(len(gt_tr))]
            gt_te = [img_crop(gt_te[i], patch_size, patch_size) for i in range(len(gt_te))]
            
            im_tr = np.asarray([im_tr[i][j] 
                              for i in range(len(im_tr)) 
                              for j in range(len(im_tr[i]))])
            im_te = np.asarray([im_te[i][j] 
                              for i in range(len(im_te)) 
                              for j in range(len(im_te[i]))])
            gt_tr =  np.asarray([gt_tr[i][j]
                              for i in range(len(gt_tr)) 
                              for j in range(len(gt_tr[i]))])
            gt_te =  np.asarray([gt_te[i][j] 
                              for i in range(len(gt_te)) 
                              for j in range(len(gt_te[i]))])
            
            im_tr=[extract_features(im_tr[i]) for i in range(len(im_tr))]
            im_te=[extract_features(im_te[i]) for i in range(len(im_te))]
            
            im_tr = np.asarray([poly_features(im_tr[i],deg) for i in range(len(im_tr))])
            gt_tr = np.asarray([value_to_class(np.mean(gt_tr[i])) for i in range(len(gt_tr))])

            im_te = np.asarray([poly_features(im_te[i],deg) for i in range(len(im_te))])
            gt_te = np.asarray([value_to_class(np.mean(gt_te[i])) for i in range(len(gt_te))])

            ridgereg = linear_model.Ridge(alpha=lam, normalize=True)
            ridgereg.fit(im_tr, gt_tr)

            Z_te = ridgereg.predict(im_te)
            Z_te=1*(Z_te>t)
            Z_pp=[]
            for i in range(len(gt_te)):
                Z_pp = Z_pp + post_processing(Z_te[i*625:(i+1)*625],18,9,3,3,25)

            nb_f1_te[ind]=compute_F1(gt_te, Z_pp)
            print(nb_f1_te[ind])
            ind=ind+1
        
        mean_f1[id_lam,id_t]=nb_f1_te.mean()
    
best=np.max(mean_f1)
ids=np.where(mean_f1==best)
best_lambda=lambdas[ids[0][0]]
best_thresh=thresh[ids[1][0]]

800
[array([ 2,  4,  8,  9, 10, 11, 13, 14, 15, 16, 18, 19, 21, 25, 28, 29, 30,
       33, 37, 38, 43, 44, 45, 48, 55, 56, 57, 58, 59, 63, 64, 65, 66, 68,
       69, 71, 73, 74, 76, 77, 80, 81, 82, 83, 85, 88, 91, 92, 97, 98]), array([ 0,  1,  3,  5,  6,  7, 12, 17, 20, 22, 23, 24, 26, 27, 31, 32, 34,
       35, 36, 39, 40, 41, 42, 46, 47, 49, 50, 51, 52, 53, 54, 60, 61, 62,
       67, 70, 72, 75, 78, 79, 84, 86, 87, 89, 90, 93, 94, 95, 96, 99])]
[array([ 0,  1,  3,  5,  6,  7, 12, 17, 20, 22, 23, 24, 26, 27, 31, 32, 34,
       35, 36, 39, 40, 41, 42, 46, 47, 49, 50, 51, 52, 53, 54, 60, 61, 62,
       67, 70, 72, 75, 78, 79, 84, 86, 87, 89, 90, 93, 94, 95, 96, 99]), array([ 2,  4,  8,  9, 10, 11, 13, 14, 15, 16, 18, 19, 21, 25, 28, 29, 30,
       33, 37, 38, 43, 44, 45, 48, 55, 56, 57, 58, 59, 63, 64, 65, 66, 68,
       69, 71, 73, 74, 76, 77, 80, 81, 82, 83, 85, 88, 91, 92, 97, 98])]
lambda: 0
thresh: 0
here
[ 2  4  8  9 10 11 13 14 15 16 18 19 21 25 28 29 30 33 37 38 43 44 45 48
 55 

In [ ]:
best

In [ ]:
mean_f1

In [ ]:
thres=np.linspace(0.3,0.8,11)

In [ ]:
zers=np.zeros((thres.shape[0],lambdas.shape[0]))
np.argmax(zers)

In [ ]:
thres.shape[0]

In [ ]:
matrix=np.asarray([[1,2,3],[4,5,6]])
matrix

In [ ]:
ids=np.where(matrix==3)
ids[1][0]